# Filtragem de Séries Temporais

**Bibliografia:**

- Roger D. Peng, 2020. A Very Short Course on Time Series Analysis. Disponível em https://bookdown.org/rdpeng/timeseriesbook/
- Hyndman, R.J., & Athanasopoulos, G. (2018) Forecasting: principles and practice, 2nd edition, OTexts: Melbourne, Australia. OTexts.com/fpp2. Disponível em https://otexts.com/fpp2/ 
- Lyons, R.G., 1997. Understanding digital signal processing, 3/E. Pearson Education India.

In [ ]:
import numpy as np
import pandas as pd

# Processamento de sinal/séries temporais
from scipy import signal

# Criação de gráficos
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Filtro passa baixa

Em geral, a filtragem consiste em processar um sinal no domínio do tempo para resultar alguma mudança no conteúdo de frequência do sinal original. Mudanças que normalmente são a **redução, ou filtragem de alguns componentes de frequência indesejados**. No caso de um filtro passa baixa, queremos um filtro que permita a passagem de "baixas" frequências e atenue "altas" frequências. 

Um método simples de construir um filtro, consiste em primeiramente decidir qual frequência queremos filtrar. 

In [ ]:
# Define um vetor que representa nosso filtro no domínio da frequência
discr_f = np.array([0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0])


fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(-15, 15), y=discr_f, name='discrete filter', mode='markers', 
                         marker_symbol='square', marker_color='black'))
fig.add_trace(go.Bar(x=np.arange(-15, 15), y=discr_f, width=0.05, showlegend=False, marker_color='black'))
fig.show()

Como trabalhamos com vetores, o qual não apresenta valores negativos é necessário realizar a aplicação de um shift no vetor.

In [ ]:
# Rotaciona as posições do vetor, colocando as posições negativas ao final do vetor
shift_discr_f = np.roll(discr_f, int(discr_f.shape[0]/2)+1)

fig = go.Figure()
fig.add_trace(go.Scatter(y=shift_discr_f, name='shifted filter', mode='markers', 
                         marker_symbol='square', marker_color='black'))
fig.add_trace(go.Bar(y=shift_discr_f, width=0.05, showlegend=False, marker_color='black'))
fig.show()

Dado que geralmente trabalha-se com séries temporais no domínio do tempo, e o filtro ainda se encontra no domínio da frequência, aplicamos a transformada de Fourier inversa.

In [ ]:
# Aplica um Transformada de Fourier Inversa no filtro e extrai a parte real do filtro.
shift_low_pass_f = np.real(np.fft.ifft(shift_discr_f))

fig = go.Figure()
fig.add_trace(go.Scatter(y=shift_low_pass_f, name='shfited low-pass filter'))
fig.show()

Porém, o filtro ainda se encontra com o shift, ao remove-lo, temos o nosso filtro pronto.

In [ ]:
# Remove o shift que foi aplicado ao filtro no domínio frequência
low_pass_f = np.roll(shift_low_pass_f, -int(discr_f.shape[0]/2)-1)

fig = go.Figure()
fig.add_trace(go.Scatter(y=low_pass_f, name='low-pass filter'))
fig.show()

Este método de design de FIR é conhecido como window method. Na prática, não precisamos realizar todos este processo, pois a biblioteca signal já possui um método para a criação de filtros utilizando window method.

In [ ]:
# Usa a função firwin para construir um filtro
px.line(y=signal.firwin(31, 0.28))

Aplicando o filtro de passa baixa na série temporal

In [ ]:
# Abertura do csv que contém a série temporal
daily_min_temp = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv")
daily_min_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3650 entries, 0 to 3649
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    3650 non-null   object 
 1   Temp    3650 non-null   float64
dtypes: float64(1), object(1)
memory usage: 57.2+ KB


In [ ]:
px.line(daily_min_temp, x='Date', y='Temp')

In [ ]:
# Por meio de uma convolução aplica-se o filtro construído com a função firwin
filtered_temp = np.convolve(daily_min_temp.Temp, signal.firwin(31, 0.28), mode='same')

fig = go.Figure()

fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=daily_min_temp.Temp, name='original'))
fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=filtered_temp, name='filtrado'))

fig.show()

In [ ]:
# Aplica-se o mesmo filtro filtro porém com parametros diferentes
filtered_temp = np.convolve(daily_min_temp.Temp, signal.firwin(31, 0.1), mode='same')

fig = go.Figure()

fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=daily_min_temp.Temp, name='original'))
fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=filtered_temp, name='filtrado'))

fig.show()

In [ ]:
# Inspeção visual das frequências que compõem o sinal original e filtrado.
fig = go.Figure()

fig.add_trace(go.Scatter(y=np.fft.fft(daily_min_temp.Temp).real, name='original'))
fig.add_trace(go.Scatter(y=np.fft.fft(filtered_temp).real, name='filtrado'))

fig.show()

## Filtro passa alta

Utilizando este mesmo processo podemos remover as altas frequências da nossa série temporal. Para isto, podemos simplesmente inverter o vetor que mapeia quais frequências desejamos manter na série temporal.

In [ ]:
# Construição de um array que representa o filtro no domínio da frquência
discr_f = np.array([1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1])

# Rotaciona as posições negativas para ficarem no final do vetor
shift_discr_f = np.roll(discr_f, int(discr_f.shape[0]/2)+1)

# Traz o filtro do domínio da frequência para o domínio da série temporal
shift_high_pass_f = np.real(np.fft.ifft(shift_discr_f))

# Remove o deslocamento aplicado ao filtro no domínio da frequência
high_pass_f = np.roll(shift_high_pass_f, -int(discr_f.shape[0]/2)-1)


fig = make_subplots(rows=4, cols=1)

fig.add_trace(go.Scatter(y=discr_f, name='discrete filter', mode='markers', 
                         marker_symbol='square', marker_color='black'), row=1, col=1)
fig.add_trace(go.Bar(y=discr_f, width=0.05, showlegend=False, marker_color='black'), row=1, col=1)

fig.add_trace(go.Scatter(y=shift_discr_f, name='shfited discrete filter', mode='markers', 
                         marker_symbol='square', marker_color='red'), row=2, col=1)
fig.add_trace(go.Bar(y=shift_discr_f, width=0.05, showlegend=False, marker_color='red'), row=2, col=1)

fig.add_trace(go.Scatter(y=shift_high_pass_f, name='shfited high-pass filter'), row=3, col=1)
fig.add_trace(go.Scatter(y=high_pass_f, name='high-pass filter'), row=4, col=1)

fig.update_layout(width=1400, height=1000)
fig.show()

Com a mesma facilidade podemos construir o filtro utilizando a biblioteca signal.

In [ ]:
# Aplicação do filtro de passa alta construído por meio da função firwin
filtered_temp = np.convolve(daily_min_temp.Temp, signal.firwin(31, 0.15, pass_zero=False), mode='same')

fig = go.Figure()

fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=daily_min_temp.Temp, name='original'))
fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=filtered_temp, name='filtrado'))

fig.show()

A versão filtrada da série temporal pode ser vista como o resíduo da série com as tendências removidas. O que pode ser útil em determinadas situações!

In [ ]:
# Inspeção visual das frequências que compõem o sinal original e filtrado.
fig = go.Figure()

fig.add_trace(go.Scatter(y=np.fft.fft(daily_min_temp.Temp).real, name='original'))
fig.add_trace(go.Scatter(y=np.fft.fft(filtered_temp).real, name='filtrado'))

fig.show()

## Filtro de Média

O filtro de média, também conhecido com média móvel, possui uma função similar ao filtro de passa baixa, podendo ser classificado como tal. A média móvel é muito utilizada para remover medições randômicas/ruídos, estimar ciclos e tendências, além de ser um método clássico de decomposição de séries temporais.

Matematicamente uma média móvel de ordem $m$ pode ser escrita como:

$$
T_t = \frac{1}{m} \sum_{j=-k}^{k} y_{t+j}
$$

Onde $m = 2k+1$

In [ ]:
# Construção do filtro de média
kernel_size = 31 # janela com tamanho "mensal" (ou quase)
kernel = np.ones(kernel_size) / kernel_size

# Aplicação do filtro de média na série temporal
filtered_temp = np.convolve(daily_min_temp.Temp, kernel, mode='same') 

fig = go.Figure()

fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=daily_min_temp.Temp, name='original'))
fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=filtered_temp, name='MA-31'))

fig.show()

In [ ]:
# Construção do filtro de média
kernel_size = 93 # janela "trimestral"
kernel = np.ones(kernel_size) / kernel_size

# Aplicação do filtro de média na série temporal
filtered_temp2 = np.convolve(daily_min_temp.Temp, kernel, mode='same') 


# Construção do filtro de média
kernel_size = 185 # janela "semestral"
kernel = np.ones(kernel_size) / kernel_size

# Aplicação do filtro de média na série temporal
filtered_temp3 = np.convolve(daily_min_temp.Temp, kernel, mode='same') 


fig = go.Figure()

fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=filtered_temp, name='MA-31'))
fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=filtered_temp2, name='MA-93'))
fig.add_trace(go.Scatter(x=daily_min_temp.Date, y=filtered_temp3, name='MA-185'))

fig.show()

In [ ]:
# Comparação dos efeitos do filtro de média e passa baixa no domínio da frequência.
fig = go.Figure()

fig.add_trace(go.Scatter(y=np.fft.fft(daily_min_temp.Temp).real, name='original'))
fig.add_trace(go.Scatter(y=np.fft.fft(filtered_temp).real, name='MA-31'))
fig.add_trace(go.Scatter(y=np.fft.fft(np.convolve(daily_min_temp.Temp, signal.firwin(31, 0.1), mode='same')).real, name='low-pass'))

fig.show()

## Mediana

Assim como utilizamos a média como filtro, podemos também a utilizar a mediana. O filtro da mediana é muito utilizado para reduzir a presença de ruídos/valores anômalos.

In [ ]:
# Abertura do csv que contém a série temporal
sunspots = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv")
sunspots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2820 entries, 0 to 2819
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Month     2820 non-null   object 
 1   Sunspots  2820 non-null   float64
dtypes: float64(1), object(1)
memory usage: 44.2+ KB


In [ ]:
px.line(sunspots, x='Month', y='Sunspots')

In [ ]:
# Aplicação de um filtro de mediana utilizando a função medfilt
filtered_temp = signal.medfilt(sunspots.Sunspots, 3)

fig = go.Figure()

fig.add_trace(go.Scatter(x=sunspots.Month, y=sunspots.Sunspots, name='original'))
fig.add_trace(go.Scatter(x=sunspots.Month, y=filtered_temp, name='MM-3'))

fig.show()

Comparando com o filtro de média...

In [ ]:
# Construção do filtro de média
kernel_size = 3
kernel = np.ones(kernel_size) / kernel_size

# Aplicação do filtro na série temporal
filtered_temp2 = np.convolve(sunspots.Sunspots, kernel, mode='same')

In [ ]:
# Comparando os efeitos do filtro de média e mediana no domínio da frequência
fig = go.Figure()

fig.add_trace(go.Scatter(y=np.fft.fft(sunspots.Sunspots).real, name='original'))
fig.add_trace(go.Scatter(y=np.fft.fft(filtered_temp).real, name='MM-3'))
fig.add_trace(go.Scatter(y=np.fft.fft(filtered_temp2).real, name='MA-3'))

fig.show()